In [62]:
# sample reader to read samples from /data/runs/samples
# and build each sample into a matrix

# Isaac Berez
# 14.01.23

from scipy.io import mmread
import os
import glob
import pandas as pd
import numpy as np
from pandas_ods_reader import read_ods

In [63]:
#get meta data from ods file, only will need sample id, sex, group
analysis_dir = '/home/isaac/analysis/scRNA-seq/'
dimorph_sample_file = 'Dimorph_samples.ods'
dimorph_df = read_ods(analysis_dir+dimorph_sample_file)
dimorph_df.head()

,Serial_Number:int,Date_Captured:string,Species:string,Transcriptome:string,Strain:string,Project:string,Group,ChipID:string,SampleID:string,DonorID:string,...,Comments:string,cDNA_Lib_Ok:string,ngperul_cDNA:float64,Avesizebp_cDNAlib,Date,cDNAul,LIbConstructionComment,ngperul_seqlib:float64,lengthbp_seqlib:float64,Sample_Index:string
0,106.0,43993.0,Mm,Mm10,Cntnp-C57Bl/6,Cntnp_KO,Cntnp-KO-F,10X54,10X54_1,Cntnp-KO_2-9F,...,"Cntnp-KO female, showing Cntnp ""male"" ASD emot...",None,24.7,NaN,None,NaN,None,17.9,473.0,B3
1,107.0,43993.0,Mm,Mm10,Cntnp-C57Bl/6,Cntnp_KO,Cntnp-KO-F,10X54,10X54_2,Cntnp-KO_4-10F,...,"Cntnp-KO female, showing Cntnp ""male"" ASD emot...",None,37.7,NaN,None,NaN,None,20.0,471.0,D3
2,212.0,2021-03-22,Mm,Mm10,Cntnp-C57Bl/6,Cntnp_KO,Cntnp-KO-M,10x98,10x98_2,Cntnp-KO_2_M,...,"v3! Cntnap KO #2, M; behavioral testing previo...",None,39.7,1873.5,2021-04-04,5.0,Single index; scale down; PCR_14 cycles,17.4,479.2,H9
3,213.0,2021-03-22,Mm,Mm10,Cntnp-C57Bl/6,Cntnp_KO,Cntnp-KO-M,10x98,10x98_3,Cntnp-KO_3_M,...,"v3! Cntnap KO #3, M; behavioral testing previo...",None,45.5,1841.7,2021-04-04,5.0,Single index; scale down; PCR_14 cycles,21.9,426.4,A10
4,98.0,05/31/20,Mm,Mm10,C57Bl/6,Dimorph,Breeder-F,10X51,10X51_2,DI-B1-F,...,"Breeders B1, 8pups DOB 27.5 (P4). perfusion aC...",None,29.8,NaN,None,NaN,None,22.2,474.0,A2


In [82]:
list(dimorph_df.columns)

['Serial_Number:int',
 'Date_Captured:string',
 'Species:string',
 'Transcriptome:string',
 'Strain:string',
 'Project:string',
 'Group',
 'ChipID:string',
 'SampleID:string',
 'DonorID:string',
 'Age:string',
 'Num_Pooled_Animals:string',
 'Sex:string',
 'Tissue:string',
 'Cell_Conc:int',
 'Target_Num_Cells:int',
 'PCR_Cycles:int',
 'Comments:string',
 'cDNA_Lib_Ok:string',
 'ngperul_cDNA:float64',
 'Avesizebp_cDNAlib',
 'Date',
 'cDNAul',
 'LIbConstructionComment',
 'ngperul_seqlib:float64',
 'lengthbp_seqlib:float64',
 'Sample_Index:string']

In [92]:
type(dimorph_df['LIbConstructionComment'][2])

str

In [94]:
#fill in missing data types
dimorph_df_1 = dimorph_df.rename(columns={"Group":"Group:string", 
                                          "Avesizebp_cDNAlib":"Avesizebp_cDNAlib:float64",
                                          "Date":"Date:string",
                                          "cDNAul":"cDNAul:float64",
                                          "LIbConstructionComment": "LIbConstructionComment:string"
                                         })
list(dimorph_df_1.columns)

['Serial_Number:int',
 'Date_Captured:string',
 'Species:string',
 'Transcriptome:string',
 'Strain:string',
 'Project:string',
 'Group:string',
 'ChipID:string',
 'SampleID:string',
 'DonorID:string',
 'Age:string',
 'Num_Pooled_Animals:string',
 'Sex:string',
 'Tissue:string',
 'Cell_Conc:int',
 'Target_Num_Cells:int',
 'PCR_Cycles:int',
 'Comments:string',
 'cDNA_Lib_Ok:string',
 'ngperul_cDNA:float64',
 'Avesizebp_cDNAlib:float64',
 'Date:string',
 'cDNAul:float64',
 'LIbConstructionComment:string',
 'ngperul_seqlib:float64',
 'lengthbp_seqlib:float64',
 'Sample_Index:string']

In [96]:
list(dimorph_df_1)[0][":*"]

TypeError: string indices must be integers, not 'str'

In [78]:
samples = [x for x in list(dimorph_df['SampleID:string']) if x != None]
#samples

In [77]:
meta_data_dict = dict.fromkeys(samples)
meta_data_dict

{'10X54_1': None,
 '10X54_2': None,
 '10x98_2': None,
 '10x98_3': None,
 '10X51_2': None,
 '10X51_1': None,
 '10X52_1': None,
 '10X52_2': None,
 '10X51_3': None,
 '10X51_4': None,
 '10X52_3': None,
 '10X52_4': None,
 '10X35_1': None,
 '10X35_2': None,
 '10X38_1': None,
 '10X38_2': None,
 '10X36_1': None,
 '10X36_2': None,
 '10X37_1': None,
 '10X37_2': None}

In [80]:
type(dimorph_df['Transcriptome:string'][0])

str

In [68]:

sample_dir = '/data/runs/samples/'
dimorph_samples = samples

os.chdir(sample_dir)
cwd = os.getcwd()
#print (cwd)
#for loop would go here to loop over samples
dir = str((glob.glob(cwd+'/'+dimorph_samples[0]+
                 '/'+'*out*'+'/'+'outs'+
                 '/'+'filtered_feature_bc_matrix'))).replace('[','').replace(']','')[1:-1]

#print(cwd)
#print (type(dir))
#print (os.listdir(dir))

m = mmread(dir + '/' + 'matrix.mtx')


In [69]:
m_arr = m.toarray()
print (m_arr)
print (m_arr.shape)
print (type(m_arr))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(27998, 6229)
<class 'numpy.ndarray'>


In [70]:
barcodes = pd.read_csv(dir+'/'+'barcodes.tsv', sep='\t', header=None)
print(barcodes)

                       0
0     AAACCCACAACAGTGG-1
1     AAACCCACATGGCCCA-1
2     AAACCCAGTCCCTGAG-1
3     AAACGAACACTACAGT-1
4     AAACGAATCCCAGCGA-1
...                  ...
6224  TTTGGTTTCATCTGTT-1
6225  TTTGGTTTCTGTGCTC-1
6226  TTTGTTGAGATGTTCC-1
6227  TTTGTTGCATAGCACT-1
6228  TTTGTTGTCCGCGGAT-1

[6229 rows x 1 columns]


In [71]:
cells = barcodes.iloc[:,0]
print(cells)
cells = np.reshape(cells,(1,cells.size))
print (cells.shape)
#TODO - append sample ID to all cells of a given sample

0       AAACCCACAACAGTGG-1
1       AAACCCACATGGCCCA-1
2       AAACCCAGTCCCTGAG-1
3       AAACGAACACTACAGT-1
4       AAACGAATCCCAGCGA-1
               ...        
6224    TTTGGTTTCATCTGTT-1
6225    TTTGGTTTCTGTGCTC-1
6226    TTTGTTGAGATGTTCC-1
6227    TTTGTTGCATAGCACT-1
6228    TTTGTTGTCCGCGGAT-1
Name: 0, Length: 6229, dtype: object
(1, 6229)


In [72]:
features = pd.read_csv(dir+'/'+'features.tsv', sep='\t', header=None)
print(features)
print(features.iloc[0:4,1])

                        0               1                2
0      ENSMUSG00000051951            Xkr4  Gene Expression
1      ENSMUSG00000089699          Gm1992  Gene Expression
2      ENSMUSG00000102343         Gm37381  Gene Expression
3      ENSMUSG00000025900             Rp1  Gene Expression
4      ENSMUSG00000109048             Rp1  Gene Expression
...                   ...             ...              ...
27993  ENSMUSG00000079808      AC168977.1  Gene Expression
27994  ENSMUSG00000095041            PISD  Gene Expression
27995  ENSMUSG00000063897           DHRSX  Gene Expression
27996  ENSMUSG00000096730        Vmn2r122  Gene Expression
27997  ENSMUSG00000095742  CAAA01147332.1  Gene Expression

[27998 rows x 3 columns]
0       Xkr4
1     Gm1992
2    Gm37381
3        Rp1
Name: 1, dtype: object


In [73]:
gene_labels = features.iloc[:,1]
gene_labels = np.reshape(gene_labels ,(gene_labels.size,1))
gene_labels.shape

(27998, 1)

In [74]:
gene_labels

array([['Xkr4'],
       ['Gm1992'],
       ['Gm37381'],
       ...,
       ['DHRSX'],
       ['Vmn2r122'],
       ['CAAA01147332.1']], dtype=object)

In [75]:
list(cells)

[array(['AAACCCACAACAGTGG-1', 'AAACCCACATGGCCCA-1', 'AAACCCAGTCCCTGAG-1',
        ..., 'TTTGTTGAGATGTTCC-1', 'TTTGTTGCATAGCACT-1',
        'TTTGTTGTCCGCGGAT-1'], dtype=object)]

In [76]:
#combine into single dataframe
sample_df = pd.DataFrame(data = m_arr,
                         index = gene_labels,
                         columns = list(cells))
sample_df.head()

,AAACCCACAACAGTGG-1,AAACCCACATGGCCCA-1,AAACCCAGTCCCTGAG-1,AAACGAACACTACAGT-1,AAACGAATCCCAGCGA-1,AAACGAATCTGCTTAT-1,AAACGCTAGCAGATAT-1,AAACGCTAGTGCTCAT-1,AAACGCTGTCCGTACG-1,AAACGCTGTGGCTACC-1,...,TTTGGTTAGAGTCACG-1,TTTGGTTCACAGCCTG-1,TTTGGTTCAGCCTTCT-1,TTTGGTTGTCACTAGT-1,TTTGGTTGTTCCTAAG-1,TTTGGTTTCATCTGTT-1,TTTGGTTTCTGTGCTC-1,TTTGTTGAGATGTTCC-1,TTTGTTGCATAGCACT-1,TTTGTTGTCCGCGGAT-1
"(Xkr4,)",0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
"(Gm1992,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(Gm37381,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(Rp1,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(Rp1,)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
